In [37]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
import multiprocessing as mp
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
base = r"D:\W\VS\VS Folder\DFD\DFD-T"

In [31]:
valid_train_df = pd.read_csv(rf"{base}\valid_train_df.csv")
# Add index to DataFrame
print(valid_train_df.head())

         filename  label  split        original  \
0  aagfhgtpmv.mp4      1  train  vudstovrck.mp4   
1  aapnvogymq.mp4      1  train  jdubbvfswz.mp4   
2  abarnvbtwb.mp4      0  train             NaN   
3  abqwwspghj.mp4      1  train  qzimuostzz.mp4   
4  acifjvzvpm.mp4      1  train  kbvibjhfzo.mp4   

                                                path  
0  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aag...  
1  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aap...  
2  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aba...  
3  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\abq...  
4  D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\aci...  


In [22]:
# # Analysis of values

# label_counts = valid_train_df['label'].value_counts()
# label_0_count = label_counts.get(0, 0)
# label_1_count = label_counts.get(1, 0)
# print(f"Label 0 count: {label_0_count}, Label 1 count: {label_1_count}")

In [23]:
row=valid_train_df.iloc[192]
print(row['path'],',', row['label'])
path= row['path']
label = row['label']

D:\W\VS\VS Folder\DFD\DFDC MTCNN Extracted\cizlkenljw , 0


In [38]:
model_path = rf'{base}\swin_tiny.pth'

In [39]:
# Load the model and processor

processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224", num_labels=1, ignore_mismatched_sizes=True)
# classifier = nn.Linear(768, 1)
criterion = nn.BCEWithLogitsLoss(reduction='none')  # For binary classification


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
# Load weights

model.load_state_dict(torch.load(model_path))
# classifier.load_state_dict(torch.load('vit_classifier_head.pth'))
model = model.to(device)
# classifier = classifier.to(device)




In [42]:
batch_size = 36
pred_classes = []
total_batches = 0

losses = []
probabs = []
correct = 0
total = 0
total_batches = 0

image_files = sorted([ os.path.join(path, f)
for f in os.listdir(path)
if f.lower().endswith('.png')
])


for i in tqdm(range(0, len(image_files), batch_size), desc='Image Batches', leave=False):
    total_batches += 1
    batch_paths = image_files[i:i+batch_size]
    images = [Image.open(p).convert("RGB") for p in batch_paths]
    inputs = processor(images=images, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    batch_labels = torch.tensor([label] * len(images), dtype=torch.float).unsqueeze(1).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        batch_logits = outputs.logits

        # running_loss += criterion(batch_logits, batch_labels).item()* batch_labels.size(0)
        loss = criterion(batch_logits, batch_labels).detach().cpu().view(-1).tolist()
        losses.extend(loss)

        prob=torch.sigmoid(batch_logits).detach().cpu().view(-1).tolist()
        probabs.extend(prob)

        pred = (torch.sigmoid(batch_logits) > 0.5).int()
        correct += (pred == batch_labels.int()).sum().item()
        total += batch_labels.size(0)
        pred_classes.extend(pred.cpu().numpy().flatten().tolist())
        

accuracy = correct / total if total > 0 else 0
val_loss = sum(losses) / total if total > 0 else 0


pred = int(sum(pred_classes) > len(pred_classes) / 2)
display(f"Predicted class: {pred}, Actual class: {label}")

Image Batches:   0%|          | 0/9 [00:00<?, ?it/s]

'Predicted class: 0, Actual class: 0'

In [ ]:
# OLD CODE IGNORE


# batch_size = 36
# pred_classes = []
# total_batches = 0

# image_files = sorted([ os.path.join(path, f)
# for f in os.listdir(path)
# if f.lower().endswith('.png')
# ])

# for i in tqdm(range(0, len(image_files), batch_size), desc='Image Batches', leave=False):
#     total_batches += 1
#     batch_paths = image_files[i:i+batch_size]
#     images = [Image.open(p).convert("RGB") for p in batch_paths]
#     inputs = processor(images=images, return_tensors="pt")
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     with torch.no_grad():
#         outputs = model(**inputs)
#         cls_embeddings = outputs.last_hidden_state[:, 0, :]  # (batch, hidden_dim)
#         outputs = classifier(cls_embeddings)  # (batch, 1)
#         preds = (torch.sigmoid(outputs) > 0.5).int()
#         pred_classes.extend(preds.cpu().numpy().flatten().tolist())

# pred = int(sum(pred_classes) > len(pred_classes) / 2)
# display(f"Predicted class: {pred}, Actual class: {label}")

